In [ ]:
imort sys
import os
import numpy as np
import matplotlib.pyplot as plt
import importlib as il
from hypso import Hypso1, Hypso2
import src.deh as deh

HYPSO_HEIGHT    =1092
HYPSO_WIDTH     =598
HYPSO_BANDS     =112 

In [ ]:
HYPSO_IMAGE_DIR = r"D:\Downloads"
DATA_DIR        = r"D:\Hierarchical Unmixing Label\hUH\data"
IMAGES_DIR      = DATA_DIR + r"\images"
DEH_DIR         = DATA_DIR + r"\deh_models"
LABELS_DIR      = DATA_DIR + r"\labels"
TREE_DIR        = DATA_DIR + r"\tree_models"
SVM_DIR    =r'\\wsl.localhost\Ubuntu-24.04\home\lofty\CODE\onboard-pipeline-modules\ground_training'

In [ ]:
image_file='vancouver_2025-05-04T19-12-24Z-l1a_flat_L1D_112_MACHI.npy'
label_file="vancouver_2025-05-04T19-12-24Z-l1a_flat_L1D_112_MACHI16end_labels.npy"
image_path=os.path.join(IMAGES_DIR, image_file)
image=np.load(image_path)

In [ ]:
hUH_model='10img_16end_L1D_112_MACHI_stabelized_aa.h5'
hUH_model_path=(os.path.join(DEH_DIR, hUH_model))

### Plot level n labels for hUH label file(.npy) 

In [ ]:
def plot_level_labels(labels_filename, level=None):
    """
    Visualizes binary labels for hierarchical nodes at a specified level.
    Highlights overlapping regions between different node labels.
    
    Args:
        labels_filename (str): Name of the .npy file containing binary labels
        level (int, optional): Hierarchical level to plot. If None, uses maximum level.
    """
    # Load and prepare data
    labels = np.load(os.path.join(LABELS_DIR, labels_filename), allow_pickle=True).item()
    max_level = max(len(k) for k in labels.keys())
    level = max_level if level is None or level > max_level else level
    
    # Get nodes at specified level
    level_keys = [k for k in labels.keys() if len(k) == level]
    if not level_keys:
        print(f"No nodes found at level {level}")
        return

    # Setup visualization
    plt.figure(figsize=(10, 10))
    first_key = level_keys[0]
    combined_labels = np.zeros(labels[first_key].reshape(-1, HYPSO_HEIGHT).shape)
    
    # Calculate overlaps
    for key in level_keys:
        combined_labels += labels[key].reshape(-1, HYPSO_HEIGHT)
    has_overlaps = np.any(combined_labels > 1)
    
    # Plot individual node labels
    for i, key in enumerate(level_keys):
        plot_label = labels[key].reshape(-1, HYPSO_HEIGHT)
        plot_label = np.flipud(np.rot90(plot_label))
        colors = [(1,1,1,0), plt.cm.viridis(i/len(level_keys))]
        node_cmap = plt.matplotlib.colors.LinearSegmentedColormap.from_list(f'custom_viridis_{i}', colors)
        plt.imshow(plot_label, aspect=0.1, alpha=0.5 if has_overlaps else 1.0, cmap=node_cmap)
    
    # Highlight overlapping regions
    if has_overlaps:
        overlap_display = np.rot90((combined_labels > 1).astype(float))
        plt.imshow(overlap_display, aspect=0.1, alpha=0.7, cmap='Reds', label='Overlapping Regions')
    
    # Configure plot
    plt.title(f'Binary Labels for Level {level} Nodes')
    legend_elements = [plt.Rectangle((0,0),1,1, facecolor=plt.cm.viridis(i/len(level_keys))) 
                      for i in range(len(level_keys))]
    if has_overlaps:
        legend_elements.append(plt.Rectangle((0,0),1,1, facecolor='red', alpha=0.7))
        legend_labels = list(level_keys) + ['Overlaps']
    else:
        legend_labels = list(level_keys)
    plt.legend(legend_elements, legend_labels, loc='center left', bbox_to_anchor=(1, 0.5))
    plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_level_labels(label_file, level=7)

### Plot level n spectra for hUH model(.h5) and image (.npy)

In [ ]:
def plot_spectra_for_level(save_file_path, cm_input, level_input=None):
    """
    Plot spectral signatures for nodes at a specified level from a DEH model.
    
    Args:
        save_file_path: Path to saved DEH model
        cm_input: Input data cube
        level_input: Hierarchical level to display (defaults to deepest level)
    """
    DEH_temp = deh.DEH(no_negative_residuals=True)
    DEH_temp.load(save_file_path)
    cube_image=cm_input.reshape(-1,HYPSO_HEIGHT,HYPSO_BANDS)

    DEH_temp.plot_size = (cube_image.shape[0],cube_image.shape[1])
    DEH_temp.simple_predict(cm_input)
    nodes = DEH_temp.nodes
    num_levels = max(len(node) for node in DEH_temp.nodes)
    
    # If level_input is empty or invalid, use the last level
    if level_input == None or not isinstance(level_input, int) or level_input > num_levels or level_input < 1:
        level = num_levels
    else:
        level = level_input
        
    # Get nodes at specified level
    level_nodes = [node for node in nodes if len(node) == level]
    print("using level", level, "with", len(level_nodes), "nodes")
    DEH_temp.display_spectra(level_nodes)

In [ ]:
plot_spectra_for_level(hUH_model_path, image, level_input=7)